Тестовое задание
БД postgresql
Таблица оплат "test_task_billing":
==============

Таблица партнеров "test_task_partner_acc":
================
"partnerId" Имя партнера
"accountId" Аккаунт партнера

Задание:
1.Посчитать выручку за каждый месяц (SQL, решение в блокноте)

2.Посчитать сумму пополнений клиентов партнеров по годам и какой это процент от общей выручки в этот же год (SQL, решение в блокноте). Поля год, партнер, выручка, процент от общей выручки по всем клиентам в этот год.

3.Постройте график выручки по месяцам (как удобней)

In [ ]:
import psycopg2
import pandas as pd
import plotly.graph_objects as go

In [ ]:
connection = psycopg2.connect(user="xxxxx",
                                  password="xxxxx",
                                  host="168.119.252.xxxxx",
                                  port="5432",
                                  database="xxxxx_data_mart")

In [ ]:
cursor = connection.cursor()

1. Посчитать выручку за каждый месяц (SQL, решение в блокноте)

In [ ]:
select_query = '''SELECT ABS(SUM(transaction)) AS revenue, date_trunc('month', datetime) AS month   
FROM test_task_billing WHERE object IN ('channel', 'package')
GROUP BY  month'''
cursor.execute(select_query)

In [ ]:
tupples = cursor.fetchall()
column_names = ['revenue', 'month']    
df = pd.DataFrame(tupples, columns=column_names)

In [ ]:
df

,revenue,month
0,38.400,2018-01-01
1,612.000,2018-02-01
2,8364.000,2018-03-01
3,11887.200,2018-04-01
4,17020.800,2018-05-01
5,21211.200,2018-06-01
6,17191.200,2018-07-01
7,16284.000,2018-08-01
8,14162.400,2018-09-01
9,20800.800,2018-10-01


3. Постройте график выручки по месяцам (как удобней)

In [ ]:
fig = go.Figure(data=go.Scatter(x=df.month, y=df.revenue, mode ='lines+markers'))
fig.update_layout(title='Revenue per month',
                   xaxis_title='Month',
                   yaxis_title='Revenue')
fig.show()

2. Посчитать сумму пополнений клиентов партнеров по годам и какой это процент от общей выручки в этот же год (SQL, решение в блокноте). Поля год, партнер, выручка, процент от общей выручки по всем клиентам в этот год.

In [ ]:
cursor = connection.cursor()

In [ ]:
select_query = '''SELECT year, partnerid, revenue,  (revenue/year_revenue * 100) as ratio FROM
(SELECT date_part('year', datetime) AS year, partnerid, ABS(SUM(transaction)) AS revenue, 2 as one FROM test_task_billing
LEFT JOIN test_task_partner_acc USING (accountid)
WHERE object IN ('channel', 'package')
GROUP BY year, partnerid
ORDER BY year, partnerid) as l
JOIN 
(SELECT date_part('year', datetime) AS year,  ABS(SUM(transaction)) AS year_revenue FROM test_task_billing
LEFT JOIN test_task_partner_acc USING (accountid)
WHERE object IN ('channel', 'package')
GROUP BY year
ORDER BY year) AS r USING (year)'''
cursor.execute(select_query) 

In [ ]:
tupples = cursor.fetchall()
column_names = ['year', 'partnerid', 'revenue', 'ratio']    
fdf = pd.DataFrame(tupples, columns=column_names)

In [ ]:
fdf

,year,partnerid,revenue,ratio
0,2018.0,10212.0,4.80,0.002504
1,2018.0,10288.0,7.20,0.003755
2,2018.0,11195.0,288.00,0.150210
3,2018.0,11867.0,4.80,0.002504
4,2018.0,12146.0,14.40,0.007511
...,...,...,...,...
1965,2021.0,99569.0,360.00,0.009016
1966,2021.0,99752.0,4320.00,0.108192
1967,2021.0,99762.0,10800.00,0.270481
1968,2021.0,99945.0,2340.00,0.058604
